In [1]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
# from google.colab import drive
# drive.mount("/content/gdrive")
pre_path = "" # 'C:\Users\Tugrul\Desktop\Dersler\ANN & DL\HW2'

In [2]:
# Installing packages:
# pip install pandas
# python -m pip install -U matplotlib
# python3 -m pip install tensorflow[and-cuda]
# pip install -U scikit-learn
# pip install seaborn
# pip install opencv-python
# pip install opencv-python-headless
# pip install statsmodels
# gdown --no-check-certificate --folder <FOLDER URL>

In [5]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
seed = 42
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
import numpy as np
np.random.seed(seed)
import logging
import random
random.seed(seed)
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)
#from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
import cv2
from statsmodels.tsa.stattools import acf
#%matplotlib widget

2.15.0


In [6]:
def rolling_window(array, window_shape, stride=1):
    array = np.array(array)
    shape = (array.shape[0] - window_shape + 1, window_shape)
    strides = (array.strides[0],) + array.strides
    rolled = np.lib.stride_tricks.as_strided(array, shape=shape, strides=strides)
    return rolled[np.arange(0, shape[0], stride)]

In [7]:
datas = np.load(pre_path + 'training_dataset/training_data.npy', allow_pickle=True)
indexes = np.load(pre_path + 'training_dataset/valid_periods.npy', allow_pickle=True)
labels = np.load(pre_path + 'training_dataset/categories.npy', allow_pickle=True)

In [8]:
categories = ['A', 'B', 'C', 'D', 'E', 'F']
window_sizes = {'A':200, 'B':200, 'C':200, 'D':200, 'E':200, 'F':200}
output_window_sizes = {'A':18, 'B':18, 'C':18, 'D':18, 'E':18, 'F':18}
encodings = {'A':np.array([1,0,0,0,0,0]).reshape(1,-1),
            'B':np.array([0,1,0,0,0,0]).reshape(1,-1),
            'C':np.array([0,0,1,0,0,0]).reshape(1,-1),
            'D':np.array([0,0,0,1,0,0]).reshape(1,-1),
            'E':np.array([0,0,0,0,1,0]).reshape(1,-1),
            'F':np.array([0,0,0,0,0,1]).reshape(1,-1),
           }


validation_sizes = {'A':0.2, 'B':0.2, 'C':0.2, 'D':0.2, 'E':0.2, 'F':0.2}
test_sizes = {'A':0.1, 'B':0.1, 'C':0.1, 'D':0.1, 'E':0.1, 'F':0.1}




X_train = {}
y_train = {}

X_validation = {}
y_validation = {}

X_test = {}
y_test = {}

stride = 1

In [9]:
for i,category in enumerate(categories):
    label_filter = labels == category
    batch_data = datas[label_filter]
    batch_index = indexes[label_filter]
    N = batch_data.shape[0]
    window = window_sizes[category]
    output_window_size = output_window_sizes[category]

    #--------------------------------------------------------
    X_test[category] = []
    y_test[category] = []

    X_validation[category] = []
    y_validation[category] = []

    X_train[category] = []
    y_train[category] = []
    #--------------------------------------------------------

    encode = encodings[category]
    test_end = int(N * test_sizes[category])
    validation_end = test_end + int(N * validation_sizes[category])
    train_end = N


    for n in range(N):
        start = batch_index[n,0]
        end = batch_index[n,1]

        if (end - start) < (window + output_window_size + 18):
            start_ = end - (window + output_window_size + 18)
        else:
            start_ = start

        x_n = pd.Series(batch_data[n,start_:end])
        y_n = rolling_window(x_n[window:],output_window_size,stride)


        x_n = rolling_window(x_n,window,stride)[:y_n.shape[0]]

        min = np.concatenate([x_n.min(1).reshape(-1,1),y_n.min(1).reshape(-1,1)],1).min(1).reshape(-1,1)
        max = np.concatenate([x_n.max(1).reshape(-1,1),y_n.max(1).reshape(-1,1)],1).max(1).reshape(-1,1)


        x_n = (x_n - min)/(max - min)
        y_n = (y_n - min)/(max - min)

        x_n = np.concatenate([x_n,np.repeat(encode,x_n.shape[0], axis=0)],axis = 1)

        #--------------------------------------------------------
        if n < test_end:
            X_test[category].append(x_n)
            y_test[category].append(y_n)
        elif test_end <= n < validation_end:
            X_validation[category].append(x_n)
            y_validation[category].append(y_n)
        else:
            X_train[category].append(x_n)
            y_train[category].append(y_n)
        #--------------------------------------------------------

    #--------------------------------------------------------
    X_test[category] = np.concatenate(X_test[category])
    y_test[category] = np.concatenate(y_test[category])

    X_validation[category] = np.concatenate(X_validation[category])
    y_validation[category] = np.concatenate(y_validation[category])

    X_train[category] = np.concatenate(X_train[category])
    y_train[category] = np.concatenate(y_train[category])
    #--------------------------------------------------------
    print('---------------------------------------------------------------')
    print('{} train size = '.format(category),X_train[category].shape,y_train[category].shape)
    print('{} validation size = '.format(category),X_validation[category].shape,y_validation[category].shape)
    print('{} test size = '.format(category),X_test[category].shape,y_test[category].shape)



---------------------------------------------------------------
A train size =  (344064, 206) (344064, 18)
A validation size =  (93103, 206) (93103, 18)
A test size =  (41143, 206) (41143, 18)
---------------------------------------------------------------
B train size =  (264319, 206) (264319, 18)
B validation size =  (87362, 206) (87362, 18)
B test size =  (87004, 206) (87004, 18)
---------------------------------------------------------------
C train size =  (331639, 206) (331639, 18)
C validation size =  (96050, 206) (96050, 18)
C test size =  (158726, 206) (158726, 18)
---------------------------------------------------------------
D train size =  (408375, 206) (408375, 18)
D validation size =  (165116, 206) (165116, 18)
D test size =  (113722, 206) (113722, 18)
---------------------------------------------------------------
E train size =  (291826, 206) (291826, 18)
E validation size =  (121625, 206) (121625, 18)
E test size =  (83735, 206) (83735, 18)
---------------------------

In [10]:
np.save(pre_path + 'windowed_training_data/test_X.npy', X_test)
np.save(pre_path + 'windowed_training_data/test_y.npy', y_test)

np.save(pre_path + 'windowed_training_data/validation_X.npy', X_validation)
np.save(pre_path + 'windowed_training_data/validation_y.npy', y_validation)

np.save(pre_path + 'windowed_training_data/train_X.npy', X_train)
np.save(pre_path + 'windowed_training_data/train_y.npy', y_train)